In [1]:
import sys
# setting path
sys.path.append('../')

In [2]:
import cv2
from  pytorchyolo import detect, models_split_tiny
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
import torchvision.transforms as transforms
import numpy as np
from pytorchyolo.utils.utils import load_classes, rescale_boxes, non_max_suppression, print_environment_info
import pandas as pd
import time
import torch
import torchvision.ops.boxes as bops
import matplotlib.pyplot as plt
from torchvision import io as tvio

### Prepare test frames

In [3]:
video_path = "/home/fullsuper/Rex/dataset/single_car/single_car.mp4"
# video_path = "../single_car.mp4"

test_frames = []

cap = cv2.VideoCapture(video_path) # origin frame rate is 30 fps
exist_flag = True
while exist_flag:
    ret, frame = cap.read()
    if ret:
        # test_frames.append(frame)
        test_frames.append(cv2.resize(frame,(300,300),interpolation = cv2.INTER_AREA))
    
    if ret:
        exist_flag = True
    else:
        exist_flag = False
    
cap.release()

# test_frames = test_frames[190:290]

print(len(test_frames))


292


In [4]:
def convert_rgb_frame_to_tensor(image):
    img_size = 416
    # Configure input
    input_img = transforms.Compose([
    DEFAULT_TRANSFORMS,
    Resize(img_size)])(
        (image, np.zeros((1, 5))))[0].unsqueeze(0)
    input_img = input_img.cuda()

    return input_img

In [5]:
class cots_framework():

    def __init__(self, model,device) -> None:
        self.model = model
        self.device = device
        self.reference_tensor=None
        self.tensor_size= None
        self.tensor_shape = None
        self.pruning_threshold= None

        # Measurements
        self.diff_tensor_sparsity = []
        self.pruned_tensor_sparsity = []
        self.pruned_tensor_rank =[]
        self.data_size = []
        self.reconstruct_error = []
        self.head_time =[]
        self.framework_time = []
        self.tail_time = []
        self.start_event = torch.cuda.Event(enable_timing=True)
        self.end_event = torch.cuda.Event(enable_timing=True)

    def set_reference_tensor(self, test_tensor):
        head_tensor = self.model(test_tensor, 1)
        self.tensor_shape = head_tensor.shape
        self.reference_tensor = torch.zeros( self.tensor_shape, dtype=torch.float32).to(self.device)
        self.reference_tensor_edge = torch.zeros( self.tensor_shape, dtype=torch.float32).to(self.device)
        self.tensor_size = self.tensor_shape[0]*self.tensor_shape[1]*self.tensor_shape[2]*self.tensor_shape[3]

    def set_pruning_threshold(self, threshold):
        self.pruning_threshold = threshold
    
    def set_jpeg_quality(self, quality):
        self.jpeg_quality =  quality

    def calculate_snr(self,original_signal, reconstructed_signal):
        # Calculate the noise signal
        noise_signal = original_signal - reconstructed_signal
        
        # Calculate the power of the original signal
        P_signal = np.mean(np.square(original_signal))
        
        # Calculate the power of the noise signal
        P_noise = np.mean(np.square(noise_signal))
        
        # Calculate SNR in dB
        SNR_dB = 10 * np.log10(P_signal / P_noise)
        
        return SNR_dB
    
    def jpeg_encoding_decoding(self,tensor):
        # Normalize & Quantize Config
        normalize_base =torch.abs(tensor).max()
        tensor_normal = tensor / normalize_base
        scale, zero_point = (tensor_normal.max()-tensor_normal.min())/255,127
        dtype = torch.quint8

        
        tensor_normal = torch.quantize_per_tensor(tensor_normal, scale, zero_point, dtype)
        tensor_normal = tensor_normal.int_repr()
        tensor_normal = tensor_normal.to(torch.uint8).reshape((1,self.tensor_shape[1],self.tensor_shape[2]*self.tensor_shape[3]))

        # JPEG encoding/decoding
        transfer_data = tvio.encode_jpeg(tensor_normal.cpu(),self.jpeg_quality)
        self.data_size.append(transfer_data.shape[0])
        received_data = tvio.decode_jpeg(transfer_data,device="cuda")

        # Reconstruct diff tensor
        reconstructed_tensor = received_data.reshape(self.tensor_shape)
        reconstructed_tensor = (reconstructed_tensor.to(torch.float)-zero_point) * scale * normalize_base
        snr = self.calculate_snr(reconstructed_tensor.reshape(self.tensor_size).cpu().numpy() , tensor.reshape(self.tensor_size).cpu().numpy())
        self.reconstruct_error.append(snr)
        return reconstructed_tensor


    
    def object_detection(self, tensor):
        with torch.no_grad():
            # Execute head model
            self.start_event.record()
            head_tensor = self.model(tensor, 1)
            self.end_event.record()
            torch.cuda.synchronize()
            self.head_time.append(self.start_event.elapsed_time(self.end_event))

            self.start_event.record()
            # diff operator
            diff_tensor = head_tensor-self.reference_tensor
            self.diff_tensor_sparsity.append(torch.sum(diff_tensor==0).cpu().item()/self.tensor_size)

            # pruner
            diff_tensor_normal = torch.nn.functional.normalize(diff_tensor)
            pruned_tensor = diff_tensor*(abs(diff_tensor_normal) > self.pruning_threshold)
            # self.reference_tensor = self.reference_tensor + pruned_tensor
            # print(pruned_tensor.shape)
            self.pruned_tensor_sparsity.append(torch.sum(pruned_tensor==0).cpu().item()/self.tensor_size)
            self.pruned_tensor_rank.append(torch.linalg.matrix_rank(pruned_tensor).cpu().numpy().mean())

            # Compressive sensing
            # compresssed_data= self.compressive_sensing_sample(pruned_tensor)
            # reconstructed_data = self.compressive_sensing_reconstruct(compresssed_data)
            reconstructed_data  = self.jpeg_encoding_decoding(pruned_tensor)

            # recosntruct intra model tensor and inference
            # self.reference_tensor = self.reference_tensor + reconstructed_data
            self.reference_tensor = self.reference_tensor + reconstructed_data
            self.end_event.record()
            torch.cuda.synchronize()
            self.framework_time.append(self.start_event.elapsed_time(self.end_event))

            self.start_event.record()
            inference_result = self.model(self.reference_tensor,2)
            self.end_event.record()
            torch.cuda.synchronize()
            self.tail_time.append(self.start_event.elapsed_time(self.end_event))
            detection = non_max_suppression(inference_result, 0.5, 0.5)

        return detection

In [6]:
def get_iou(data,gt_df):
    label_timestamp = gt_df["frame_id"].tolist()
    iou = []
    for ts in label_timestamp:
        prediction = data.loc[data["frame_id"] == ts].reset_index()
        if len(prediction) != 1:
            iou.append(0)
        else:
            label_data = gt_df.loc[gt_df["frame_id"] == ts].reset_index()
            box2 = torch.tensor([[label_data.at[0,"xmin"], label_data.at[0,"ymin"], label_data.at[0,"xmax"], label_data.at[0,"ymax"]]], dtype=torch.float)
            box1 = torch.tensor([[prediction.at[0,"xmin"], prediction.at[0,"ymin"], prediction.at[0,"xmax"], prediction.at[0,"ymax"]]], dtype=torch.float)
            iou_temp = bops.box_iou(box1, box2)
            iou.append(iou_temp.tolist()[0][0]*100)
    return iou

def get_iou_v2(data,gt_df):
    iou = []
    label_timestamp = gt_df["frame_id"].tolist()
    for i in range(len(data)):
        if(data["frame_id"].tolist()[i] not in label_timestamp):
            iou.append(-1)
        elif (data["class_id"].tolist()[i] != 7 and data["frame_id"].tolist()[i] in label_timestamp):
            iou.append(0)
        else:
            label_data = gt_df.loc[gt_df["frame_id"] == data["frame_id"].tolist()[i]]
            prediction = data.loc[data["frame_id"] == data["frame_id"].tolist()[i]]
            box2 = torch.tensor([[label_data["xmin"].tolist()[0], label_data["ymin"].tolist()[0], label_data["xmax"].tolist()[0], label_data["ymax"].tolist()[0]]], dtype=torch.float)
            box1 = torch.tensor([[prediction["xmin"].tolist()[0], prediction["ymin"].tolist()[0], prediction["xmax"].tolist()[0], prediction["ymax"].tolist()[0]]], dtype=torch.float)
            iou_temp = bops.box_iou(box1, box2)
            iou.append(iou_temp.tolist()[0][0]*100)
    return iou

### Model test

In [7]:
# Load the YOLO model
model = models_split_tiny.load_model(
  "../config/yolov3-tiny.cfg",
  "../weights/yolov3-tiny.weights")

model.set_split_layer(7) # layer <7
model = model.eval()


In [8]:
# generated_ref = pd.read_csv("../generated_reference/split_tiny_reference.csv")
generated_ref = pd.read_csv("../generated_reference_yolov8/reference.csv")
# generated_ref = generated_ref[generated_ref["frame_id"]>=80]
# generated_ref = generated_ref[generated_ref["frame_id"]<(80+samples_number)]
generated_ref

,Unnamed: 0,frame_id,class_id,predict_acc,xmin,xmax,ymin,ymax
0,0,67,7,0.394029,270.402832,356.202393,105.519173,230.460449
1,0,70,7,0.445868,267.426453,348.659973,107.098907,223.873657
2,0,73,7,0.271297,264.173950,341.914917,103.564514,218.283112
3,0,76,7,0.374754,262.011536,335.675964,125.504120,212.531250
4,0,78,7,0.525068,260.167603,332.140259,96.572594,208.763702
...,...,...,...,...,...,...,...,...
97,0,279,7,0.431506,208.504150,252.829773,113.805267,173.270508
98,0,280,7,0.426140,208.430573,253.138275,113.497261,173.350647
99,0,281,7,0.394409,208.269165,253.282684,113.481483,173.788422
100,0,289,7,0.453462,208.941010,254.658142,112.629051,174.220673


In [9]:
car_mask = generated_ref["class_id"] ==7

refs = generated_ref[car_mask]

In [10]:
samples_number = 292

In [11]:
threshold =[]
decomposition_factor=[]
sparsity = []
iou =[]
rank = []
err = []
data_size = []
jpeg_quality = []
head_time=[]
tail_time=[]
framework_time=[]

for i in range(5):
    print("In iter",i)
    thresh = 0.05
    threshold.append(thresh)
    quality = 60+10*i
    jpeg_quality.append(quality)
    cots = cots_framework(model,device="cuda")
    cots.set_reference_tensor(convert_rgb_frame_to_tensor(test_frames[0]))
    cots.set_pruning_threshold(thresh)
    cots.set_jpeg_quality(quality)
    # cots.set_compressive_sensing(512,1024)
    # cots.set_regression_factor(factor)
    df_result = pd.DataFrame(columns=["frame_id", 'class_id',"predict_acc","xmin","xmax","ymin","ymax"])
    indexes = []
    for index in range(len(test_frames)):
    # for index in range(70,70+samples_number):
    # for index in range(1):
        print("Processing frame: ",index)
        indexes.append(index)
        frame = test_frames[index]
        frame = convert_rgb_frame_to_tensor(frame)
        detection = cots.object_detection(frame)
        # [[x1, y1, x2, y2, confidence, class]]
        if len(detection[0]!= 0):
            df2 = pd.DataFrame([[index, 
                                detection[0][0][5].item(),
                                detection[0][0][4].item(),
                                detection[0][0][0].item(),
                                detection[0][0][1].item(),
                                detection[0][0][2].item(),
                                detection[0][0][3].item()]], columns=["frame_id", 'class_id',"predict_acc","xmin","ymin","xmax","ymax"])
            df_result=pd.concat([df_result,df2])
        else:
            df2 = pd.DataFrame([[index, -1,-1,-1,-1,-1,-1]], columns=["frame_id", 'class_id',"predict_acc","xmin","ymin","xmax","ymax"])
            df_result=pd.concat([df_result,df2])

    sparsity.append(cots.pruned_tensor_sparsity)
    rank.append(cots.pruned_tensor_rank)
    iou.append(get_iou_v2(df_result,refs))
    err.append(cots.reconstruct_error)
    data_size.append(cots.data_size)
    head_time.append(cots.head_time)
    tail_time.append(cots.tail_time)
    framework_time.append(cots.framework_time)


In iter 0
Processing frame:  0
Processing frame:  1
Processing frame:  2
Processing frame:  3
Processing frame:  4
Processing frame:  5
Processing frame:  6
Processing frame:  7
Processing frame:  8
Processing frame:  9
Processing frame:  10
Processing frame:  11
Processing frame:  12
Processing frame:  13
Processing frame:  14
Processing frame:  15
Processing frame:  16
Processing frame:  17
Processing frame:  18
Processing frame:  19
Processing frame:  20
Processing frame:  21
Processing frame:  22
Processing frame:  23
Processing frame:  24
Processing frame:  25
Processing frame:  26
Processing frame:  27
Processing frame:  28
Processing frame:  29
Processing frame:  30
Processing frame:  31
Processing frame:  32
Processing frame:  33
Processing frame:  34
Processing frame:  35
Processing frame:  36
Processing frame:  37
Processing frame:  38
Processing frame:  39
Processing frame:  40
Processing frame:  41
Processing frame:  42
Processing frame:  43
Processing frame:  44
Processing

In [12]:
final_df = pd.DataFrame()

for j in range(5):

    temp_df = pd.DataFrame()
    temp_df["fid"] = np.arange(samples_number)
    temp_df["Prune"] = np.full(shape=samples_number,fill_value=threshold[j])
    temp_df["jpeg_quality"] = np.full(shape=samples_number,fill_value=jpeg_quality[j])
    temp_df["Sparsity"] = sparsity[j]
    temp_df["IoU"] = iou[j]
    temp_df["AvgRank"] = rank[j]
    temp_df["RconErr"] = err[j]
    temp_df["Data_size"] = data_size[j]
    temp_df["head_time"] = head_time[j]
    temp_df["tail_time"] = tail_time[j]
    temp_df["framework_time"] = framework_time[j]

    final_df=pd.concat([final_df,temp_df])

final_df.to_csv("../performance_test/jpeg_analysis_diff_compare_time.csv")